<a href="https://colab.research.google.com/github/Cinnamy/DocumentsClassification/blob/main/4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Подключаем модули

In [ ]:
import sklearn
import numpy as np
import nltk
import re
import pandas as pd
import torch

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn import metrics
from sklearn.datasets import fetch_20newsgroups
from transformers import BertTokenizer, BertModel, RobertaTokenizer, RobertaModel, DistilBertTokenizer, DistilBertModel

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


#Загружаем и предобрабатываем данные

In [ ]:
stopwords_en = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

Датасет с хорошо разделёнными тематиками:

In [ ]:
well_separated = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), categories=['comp.graphics', 'rec.autos', 'sci.med', 'talk.politics.mideast'])
well_separated_full = []
well_separated_target = [[], [], []]
well_separated_nouns = []
well_separated_nouns_and_adjs = []
for i in range(300):
  text = well_separated.data[i]
  text = re.sub(r'[^(\w\s\-)]|[\d]|[()]|-{2,}', '', text)
  text = re.sub(r'\s{2,}|[\n]', ' ', text)
  text = ' '.join([lemmatizer.lemmatize(word.lower()) for word in word_tokenize(text) if word not in stopwords_en])
  if text != '':
    well_separated_full.append(text)
    well_separated_target[0].append(well_separated.target[i])
  nouns = ' '.join([word[0] for word in nltk.pos_tag(word_tokenize(text)) if word[1] == "NN"])
  if nouns != '':
    well_separated_nouns.append(nouns)
    well_separated_target[1].append(well_separated.target[i])
  nouns_and_adjs = ' '.join([word[0] for word in nltk.pos_tag(word_tokenize(text)) if word[1] == "NN" or word[1] == "JJ"])
  if nouns_and_adjs != '':
    well_separated_nouns_and_adjs.append(nouns_and_adjs)
    well_separated_target[2].append(well_separated.target[i])

Датасет с плохо разделёнными тематиками:

In [ ]:
weak_separated = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), categories=['talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc'])
weak_separated_full = []
weak_separated_target = [[], [], []]
weak_separated_nouns = []
weak_separated_nouns_and_adjs = []
for i in range(300):
  text = weak_separated.data[i]
  text = re.sub(r'[^(\w\s\-)]|[\d]|-{2,}', '', text)
  text = re.sub(r'\s{2,}|[\n]', ' ', text)
  text = ' '.join([lemmatizer.lemmatize(word.lower()) for word in word_tokenize(text) if word not in stopwords_en])
  if text != '':
    weak_separated_full.append(text)
    weak_separated_target[0].append(weak_separated.target[i])
  nouns = ' '.join([word[0] for word in nltk.pos_tag(word_tokenize(text)) if word[1] == "NN"])
  if nouns != '':
    weak_separated_nouns.append(nouns)
    weak_separated_target[1].append(weak_separated.target[i])
  nouns_and_adjs = ' '.join([word[0] for word in nltk.pos_tag(word_tokenize(text)) if word[1] == "NN" or word[1] == "JJ"])
  if nouns_and_adjs != '':
    weak_separated_nouns_and_adjs.append(nouns_and_adjs)
    weak_separated_target[2].append(weak_separated.target[i])

#4 задание

##BERT

###Хорошо разделённые тематики

####Делаем векторизацию

In [ ]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

In [ ]:
well_separated_full_tokens = [tokenizer(text, padding=True, truncation=True, return_tensors="pt") for text in well_separated_full]
well_separated_nouns_tokens = [tokenizer(text, padding=True, truncation=True, return_tensors="pt") for text in well_separated_nouns]
well_separated_nouns_adjs_tokens = [tokenizer(text, padding=True, truncation=True, return_tensors="pt") for text in well_separated_nouns_and_adjs]

X_full_well = []
with torch.no_grad():
  for phrase_tokens in well_separated_full_tokens:
    outputs = model(**phrase_tokens)
    X_full_well.append(np.array(outputs.last_hidden_state.mean(dim=1).flatten()))

X_nouns_well = []
with torch.no_grad():
  for phrase_tokens in well_separated_nouns_tokens:
    outputs = model(**phrase_tokens)
    X_nouns_well.append(np.array(outputs.last_hidden_state.mean(dim=1).flatten()))

X_nouns_and_adjs_well = []
with torch.no_grad():
  for phrase_tokens in well_separated_nouns_adjs_tokens:
    outputs = model(**phrase_tokens)
    X_nouns_and_adjs_well.append(np.array(outputs.last_hidden_state.mean(dim=1).flatten()))

####Выводим метрики

In [ ]:
silhouette = []
calinski_harabasz = []
davies_bouldin = []
v_measure = []
datasets = [X_full_well, X_nouns_well, X_nouns_and_adjs_well]

for j in range(3):
  X = datasets[j]
  sum_silhouette = 0
  sum_calinski_harabasz = 0
  sum_davies_bouldin = 0
  sum_v_measure = 0
  for i in range(50):
    kmeans = KMeans(n_clusters=3, n_init=1).fit(X)
    sum_silhouette += metrics.silhouette_score(X, kmeans.labels_, metric='euclidean')
    sum_calinski_harabasz += metrics.calinski_harabasz_score(X, kmeans.labels_)
    sum_davies_bouldin += metrics.davies_bouldin_score(X, kmeans.labels_)
    sum_v_measure += metrics.v_measure_score(well_separated_target[j], kmeans.labels_)
  v_measure.append(sum_v_measure / 50)
  silhouette.append(sum_silhouette / 50)
  calinski_harabasz.append(sum_calinski_harabasz / 50)
  davies_bouldin.append(sum_davies_bouldin / 50)

kmeans_metrics = pd.DataFrame({
    "V_measure": v_measure,
    "Silhouette": silhouette,
    "Calinski-Harabasz": calinski_harabasz,
    "Davies-Bouldin": davies_bouldin
}, index=pd.Index(["Full", "Nouns", "Nouns and adjs"]))

kmeans_metrics.style.set_table_styles([{'selector': 'td', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'th', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'caption', 'props': [('font-size', '24pt')]}]).set_caption("KMeans, well separated")

,V_measure,Silhouette,Calinski-Harabasz,Davies-Bouldin
Full,0.247136,0.061353,21.453229,3.070557
Nouns,0.192320,0.067065,18.716815,3.217681
Nouns and adjs,0.266683,0.064500,19.926451,3.092692


In [ ]:
aggl_euc_metrics = [[], [], []]

for i in range(3):
  X = datasets[i]
  for link in ["ward", "complete", "average", "single"]:
    sum_silhouette = 0
    sum_calinski_harabasz = 0
    sum_davies_bouldin = 0
    sum_v_measure = 0
    for j in range(50):
      aggl = AgglomerativeClustering(n_clusters=3, linkage=link).fit(X)
      sum_silhouette += metrics.silhouette_score(X, aggl.labels_, metric='euclidean')
      sum_calinski_harabasz += metrics.calinski_harabasz_score(X, aggl.labels_)
      sum_davies_bouldin += metrics.davies_bouldin_score(X, aggl.labels_)
      sum_v_measure += metrics.v_measure_score(well_separated_target[i], aggl.labels_)
    aggl_euc_metrics[i].append(sum_v_measure / 50)
    aggl_euc_metrics[i].append(sum_silhouette / 50)
    aggl_euc_metrics[i].append(sum_calinski_harabasz / 50)
    aggl_euc_metrics[i].append(sum_davies_bouldin / 50)

aggl_euc_metrics_table = pd.DataFrame(aggl_euc_metrics,
                                index=pd.Index(["Full", "Nouns", "Nouns and adjs"]),
                                columns=pd.MultiIndex.from_product([["ward", "complete", "average", "single"],['V', 'Sil', 'Cal-Har', 'Dav-Boul']], names=['Metric', 'Linkage']))

aggl_euc_metrics_table.style.set_table_styles([{'selector': 'td', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'th', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'caption', 'props': [('font-size', '24pt')]}]).set_caption("Agglomerative clustering, Euclidean, well separated")

In [ ]:
aggl_cos_metrics = [[], [], []]

for i in range(3):
  X = datasets[i]
  for link in ["complete", "average", "single"]:
    sum_silhouette = 0
    sum_calinski_harabasz = 0
    sum_davies_bouldin = 0
    sum_v_measure = 0
    for j in range(50):
      aggl = AgglomerativeClustering(n_clusters=3, linkage=link, metric='cosine').fit(X)
      sum_silhouette += metrics.silhouette_score(X, aggl.labels_, metric='euclidean')
      sum_calinski_harabasz += metrics.calinski_harabasz_score(X, aggl.labels_)
      sum_davies_bouldin += metrics.davies_bouldin_score(X, aggl.labels_)
      sum_v_measure += metrics.v_measure_score(well_separated_target[i], aggl.labels_)
    aggl_cos_metrics[i].append(sum_v_measure / 50)
    aggl_cos_metrics[i].append(sum_silhouette / 50)
    aggl_cos_metrics[i].append(sum_calinski_harabasz / 50)
    aggl_cos_metrics[i].append(sum_davies_bouldin / 50)

aggl_cos_metrics_table = pd.DataFrame(aggl_cos_metrics,
                                index=pd.Index(["Full", "Nouns", "Nouns and adjs"]),
                                columns=pd.MultiIndex.from_product([["complete", "average", "single"],['V', 'Sil', 'Cal-Har', 'Dav-Boul']], names=['Metric', 'Linkage']))

aggl_cos_metrics_table.style.set_table_styles([{'selector': 'td', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'th', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'caption', 'props': [('font-size', '24pt')]}]).set_caption("Agglomerative clustering, cosine, well separated")

###Плохо разделённые тематики

####Делаем векторизацию

In [ ]:
weak_separated_full_tokens = [tokenizer(text, padding=True, truncation=True, return_tensors="pt") for text in weak_separated_full]
weak_separated_nouns_tokens = [tokenizer(text, padding=True, truncation=True, return_tensors="pt") for text in weak_separated_nouns]
weak_separated_nouns_adjs_tokens = [tokenizer(text, padding=True, truncation=True, return_tensors="pt") for text in weak_separated_nouns_and_adjs]

X_full_weak = []
with torch.no_grad():
  for phrase_tokens in weak_separated_full_tokens:
    outputs = model(**phrase_tokens)
    X_full_weak.append(np.array(outputs.last_hidden_state.mean(dim=1).flatten()))

X_nouns_weak = []
with torch.no_grad():
  for phrase_tokens in weak_separated_nouns_tokens:
    outputs = model(**phrase_tokens)
    X_nouns_weak.append(np.array(outputs.last_hidden_state.mean(dim=1).flatten()))

X_nouns_and_adjs_weak = []
with torch.no_grad():
  for phrase_tokens in weak_separated_nouns_adjs_tokens:
    outputs = model(**phrase_tokens)
    X_nouns_and_adjs_weak.append(np.array(outputs.last_hidden_state.mean(dim=1).flatten()))

####Выводим метрики

In [ ]:
silhouette = []
calinski_harabasz = []
davies_bouldin = []
v_measure = []
datasets = [X_full_weak, X_nouns_weak, X_nouns_and_adjs_weak]

for j in range(3):
  X = datasets[j]
  sum_silhouette = 0
  sum_calinski_harabasz = 0
  sum_davies_bouldin = 0
  sum_v_measure = 0
  for i in range(50):
    kmeans = KMeans(n_clusters=3, n_init=1).fit(X)
    sum_silhouette += metrics.silhouette_score(X, kmeans.labels_, metric='euclidean')
    sum_calinski_harabasz += metrics.calinski_harabasz_score(X, kmeans.labels_)
    sum_davies_bouldin += metrics.davies_bouldin_score(X, kmeans.labels_)
    sum_v_measure += metrics.v_measure_score(weak_separated_target[j], kmeans.labels_)
  v_measure.append(sum_v_measure / 50)
  silhouette.append(sum_silhouette / 50)
  calinski_harabasz.append(sum_calinski_harabasz / 50)
  davies_bouldin.append(sum_davies_bouldin / 50)

kmeans_metrics = pd.DataFrame({
    "V_measure": v_measure,
    "Silhouette": silhouette,
    "Calinski-Harabasz": calinski_harabasz,
    "Davies-Bouldin": davies_bouldin
}, index=pd.Index(["Full", "Nouns", "Nouns and adjs"]))

kmeans_metrics.style.set_table_styles([{'selector': 'td', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'th', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'caption', 'props': [('font-size', '24pt')]}]).set_caption("KMeans, weak separated")

,V_measure,Silhouette,Calinski-Harabasz,Davies-Bouldin
Full,0.046720,0.063066,23.509752,2.931805
Nouns,0.032603,0.064984,23.770536,3.015925
Nouns and adjs,0.053694,0.067649,24.625041,2.635518


In [ ]:
aggl_euc_metrics = [[], [], []]

for i in range(3):
  X = datasets[i]
  for link in ["ward", "complete", "average", "single"]:
    sum_silhouette = 0
    sum_calinski_harabasz = 0
    sum_davies_bouldin = 0
    sum_v_measure = 0
    for j in range(50):
      aggl = AgglomerativeClustering(n_clusters=3, linkage=link).fit(X)
      sum_silhouette += metrics.silhouette_score(X, aggl.labels_, metric='euclidean')
      sum_calinski_harabasz += metrics.calinski_harabasz_score(X, aggl.labels_)
      sum_davies_bouldin += metrics.davies_bouldin_score(X, aggl.labels_)
      sum_v_measure += metrics.v_measure_score(weak_separated_target[i], aggl.labels_)
    aggl_euc_metrics[i].append(sum_v_measure / 50)
    aggl_euc_metrics[i].append(sum_silhouette / 50)
    aggl_euc_metrics[i].append(sum_calinski_harabasz / 50)
    aggl_euc_metrics[i].append(sum_davies_bouldin / 50)

aggl_euc_metrics_table = pd.DataFrame(aggl_euc_metrics,
                                index=pd.Index(["Full", "Nouns", "Nouns and adjs"]),
                                columns=pd.MultiIndex.from_product([["ward", "complete", "average", "single"],['V', 'Sil', 'Cal-Har', 'Dav-Boul']], names=['Metric', 'Linkage']))

aggl_euc_metrics_table.style.set_table_styles([{'selector': 'td', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'th', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'caption', 'props': [('font-size', '24pt')]}]).set_caption("Agglomerative clustering, Euclidean, weak separated")

In [ ]:
aggl_cos_metrics = [[], [], []]

for i in range(3):
  X = datasets[i]
  for link in ["complete", "average", "single"]:
    sum_silhouette = 0
    sum_calinski_harabasz = 0
    sum_davies_bouldin = 0
    sum_v_measure = 0
    for j in range(50):
      aggl = AgglomerativeClustering(n_clusters=3, linkage=link, metric='cosine').fit(X)
      sum_silhouette += metrics.silhouette_score(X, aggl.labels_, metric='euclidean')
      sum_calinski_harabasz += metrics.calinski_harabasz_score(X, aggl.labels_)
      sum_davies_bouldin += metrics.davies_bouldin_score(X, aggl.labels_)
      sum_v_measure += metrics.v_measure_score(weak_separated_target[i], aggl.labels_)
    aggl_cos_metrics[i].append(sum_v_measure / 50)
    aggl_cos_metrics[i].append(sum_silhouette / 50)
    aggl_cos_metrics[i].append(sum_calinski_harabasz / 50)
    aggl_cos_metrics[i].append(sum_davies_bouldin / 50)

aggl_cos_metrics_table = pd.DataFrame(aggl_cos_metrics,
                                index=pd.Index(["Full", "Nouns", "Nouns and adjs"]),
                                columns=pd.MultiIndex.from_product([["complete", "average", "single"],['V', 'Sil', 'Cal-Har', 'Dav-Boul']], names=['Metric', 'Linkage']))

aggl_cos_metrics_table.style.set_table_styles([{'selector': 'td', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'th', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'caption', 'props': [('font-size', '24pt')]}]).set_caption("Agglomerative clustering, cosine, weak separated")

##RoBERTa

###Хорошо разделённые

####Делаем векторизацию

In [ ]:
model_name = 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaModel.from_pretrained(model_name)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
well_separated_full_tokens = [tokenizer(text, padding=True, truncation=True, return_tensors="pt") for text in well_separated_full]
well_separated_nouns_tokens = [tokenizer(text, padding=True, truncation=True, return_tensors="pt") for text in well_separated_nouns]
well_separated_nouns_adjs_tokens = [tokenizer(text, padding=True, truncation=True, return_tensors="pt") for text in well_separated_nouns_and_adjs]

X_full_well = []
with torch.no_grad():
  for phrase_tokens in well_separated_full_tokens:
    outputs = model(**phrase_tokens)
    X_full_well.append(np.array(outputs.last_hidden_state.mean(dim=1).flatten()))

X_nouns_well = []
with torch.no_grad():
  for phrase_tokens in well_separated_nouns_tokens:
    outputs = model(**phrase_tokens)
    X_nouns_well.append(np.array(outputs.last_hidden_state.mean(dim=1).flatten()))

X_nouns_and_adjs_well = []
with torch.no_grad():
  for phrase_tokens in well_separated_nouns_adjs_tokens:
    outputs = model(**phrase_tokens)
    X_nouns_and_adjs_well.append(np.array(outputs.last_hidden_state.mean(dim=1).flatten()))

####Выводим метрики

In [ ]:
silhouette = []
calinski_harabasz = []
davies_bouldin = []
v_measure = []
datasets = [X_full_well, X_nouns_well, X_nouns_and_adjs_well]

for j in range(3):
  X = datasets[j]
  sum_silhouette = 0
  sum_calinski_harabasz = 0
  sum_davies_bouldin = 0
  sum_v_measure = 0
  for i in range(50):
    kmeans = KMeans(n_clusters=3, n_init=1).fit(X)
    sum_silhouette += metrics.silhouette_score(X, kmeans.labels_, metric='euclidean')
    sum_calinski_harabasz += metrics.calinski_harabasz_score(X, kmeans.labels_)
    sum_davies_bouldin += metrics.davies_bouldin_score(X, kmeans.labels_)
    sum_v_measure += metrics.v_measure_score(well_separated_target[j], kmeans.labels_)
  v_measure.append(sum_v_measure / 50)
  silhouette.append(sum_silhouette / 50)
  calinski_harabasz.append(sum_calinski_harabasz / 50)
  davies_bouldin.append(sum_davies_bouldin / 50)

kmeans_metrics = pd.DataFrame({
    "V_measure": v_measure,
    "Silhouette": silhouette,
    "Calinski-Harabasz": calinski_harabasz,
    "Davies-Bouldin": davies_bouldin
}, index=pd.Index(["Full", "Nouns", "Nouns and adjs"]))

kmeans_metrics.style.set_table_styles([{'selector': 'td', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'th', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'caption', 'props': [('font-size', '24pt')]}]).set_caption("KMeans, well separated")

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:8

,V_measure,Silhouette,Calinski-Harabasz,Davies-Bouldin
Full,0.062791,0.125175,69.642895,2.258108
Nouns,0.016397,0.136720,54.802881,1.973646
Nouns and adjs,0.054631,0.119510,55.163932,2.254793


In [ ]:
aggl_euc_metrics = [[], [], []]

for i in range(3):
  X = datasets[i]
  for link in ["ward", "complete", "average", "single"]:
    sum_silhouette = 0
    sum_calinski_harabasz = 0
    sum_davies_bouldin = 0
    sum_v_measure = 0
    for j in range(50):
      aggl = AgglomerativeClustering(n_clusters=3, linkage=link).fit(X)
      sum_silhouette += metrics.silhouette_score(X, aggl.labels_, metric='euclidean')
      sum_calinski_harabasz += metrics.calinski_harabasz_score(X, aggl.labels_)
      sum_davies_bouldin += metrics.davies_bouldin_score(X, aggl.labels_)
      sum_v_measure += metrics.v_measure_score(well_separated_target[i], aggl.labels_)
    aggl_euc_metrics[i].append(sum_v_measure / 50)
    aggl_euc_metrics[i].append(sum_silhouette / 50)
    aggl_euc_metrics[i].append(sum_calinski_harabasz / 50)
    aggl_euc_metrics[i].append(sum_davies_bouldin / 50)

aggl_euc_metrics_table = pd.DataFrame(aggl_euc_metrics,
                                index=pd.Index(["Full", "Nouns", "Nouns and adjs"]),
                                columns=pd.MultiIndex.from_product([["ward", "complete", "average", "single"],['V', 'Sil', 'Cal-Har', 'Dav-Boul']], names=['Metric', 'Linkage']))

aggl_euc_metrics_table.style.set_table_styles([{'selector': 'td', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'th', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'caption', 'props': [('font-size', '24pt')]}]).set_caption("Agglomerative clustering, Euclidean, well separated")

In [ ]:
aggl_cos_metrics = [[], [], []]

for i in range(3):
  X = datasets[i]
  for link in ["complete", "average", "single"]:
    sum_silhouette = 0
    sum_calinski_harabasz = 0
    sum_davies_bouldin = 0
    sum_v_measure = 0
    for j in range(50):
      aggl = AgglomerativeClustering(n_clusters=3, linkage=link, metric='cosine').fit(X)
      sum_silhouette += metrics.silhouette_score(X, aggl.labels_, metric='euclidean')
      sum_calinski_harabasz += metrics.calinski_harabasz_score(X, aggl.labels_)
      sum_davies_bouldin += metrics.davies_bouldin_score(X, aggl.labels_)
      sum_v_measure += metrics.v_measure_score(well_separated_target[i], aggl.labels_)
    aggl_cos_metrics[i].append(sum_v_measure / 50)
    aggl_cos_metrics[i].append(sum_silhouette / 50)
    aggl_cos_metrics[i].append(sum_calinski_harabasz / 50)
    aggl_cos_metrics[i].append(sum_davies_bouldin / 50)

aggl_cos_metrics_table = pd.DataFrame(aggl_cos_metrics,
                                index=pd.Index(["Full", "Nouns", "Nouns and adjs"]),
                                columns=pd.MultiIndex.from_product([["complete", "average", "single"],['V', 'Sil', 'Cal-Har', 'Dav-Boul']], names=['Metric', 'Linkage']))

aggl_cos_metrics_table.style.set_table_styles([{'selector': 'td', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'th', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'caption', 'props': [('font-size', '24pt')]}]).set_caption("Agglomerative clustering, cosine, well separated")

###Плохо разделённые

####Делаем векторизацию

In [ ]:
weak_separated_full_tokens = [tokenizer(text, padding=True, truncation=True, return_tensors="pt") for text in weak_separated_full]
weak_separated_nouns_tokens = [tokenizer(text, padding=True, truncation=True, return_tensors="pt") for text in weak_separated_nouns]
weak_separated_nouns_adjs_tokens = [tokenizer(text, padding=True, truncation=True, return_tensors="pt") for text in weak_separated_nouns_and_adjs]

X_full_weak = []
with torch.no_grad():
  for phrase_tokens in weak_separated_full_tokens:
    outputs = model(**phrase_tokens)
    X_full_weak.append(np.array(outputs.last_hidden_state.mean(dim=1).flatten()))

X_nouns_weak = []
with torch.no_grad():
  for phrase_tokens in weak_separated_nouns_tokens:
    outputs = model(**phrase_tokens)
    X_nouns_weak.append(np.array(outputs.last_hidden_state.mean(dim=1).flatten()))

X_nouns_and_adjs_weak = []
with torch.no_grad():
  for phrase_tokens in weak_separated_nouns_adjs_tokens:
    outputs = model(**phrase_tokens)
    X_nouns_and_adjs_weak.append(np.array(outputs.last_hidden_state.mean(dim=1).flatten()))

####Выводим метрики

In [ ]:
silhouette = []
calinski_harabasz = []
davies_bouldin = []
v_measure = []
datasets = [X_full_weak, X_nouns_weak, X_nouns_and_adjs_weak]

for j in range(3):
  X = datasets[j]
  sum_silhouette = 0
  sum_calinski_harabasz = 0
  sum_davies_bouldin = 0
  sum_v_measure = 0
  for i in range(50):
    kmeans = KMeans(n_clusters=3, n_init=1).fit(X)
    sum_silhouette += metrics.silhouette_score(X, kmeans.labels_, metric='euclidean')
    sum_calinski_harabasz += metrics.calinski_harabasz_score(X, kmeans.labels_)
    sum_davies_bouldin += metrics.davies_bouldin_score(X, kmeans.labels_)
    sum_v_measure += metrics.v_measure_score(weak_separated_target[j], kmeans.labels_)
  v_measure.append(sum_v_measure / 50)
  silhouette.append(sum_silhouette / 50)
  calinski_harabasz.append(sum_calinski_harabasz / 50)
  davies_bouldin.append(sum_davies_bouldin / 50)

kmeans_metrics = pd.DataFrame({
    "V_measure": v_measure,
    "Silhouette": silhouette,
    "Calinski-Harabasz": calinski_harabasz,
    "Davies-Bouldin": davies_bouldin
}, index=pd.Index(["Full", "Nouns", "Nouns and adjs"]))

kmeans_metrics.style.set_table_styles([{'selector': 'td', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'th', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'caption', 'props': [('font-size', '24pt')]}]).set_caption("KMeans, weak separated")

,V_measure,Silhouette,Calinski-Harabasz,Davies-Bouldin
Full,0.049485,0.159869,90.770572,1.919464
Nouns,0.046845,0.167849,71.555307,1.744215
Nouns and adjs,0.061668,0.167758,81.826118,1.818351


In [ ]:
aggl_euc_metrics = [[], [], []]

for i in range(3):
  X = datasets[i]
  for link in ["ward", "complete", "average", "single"]:
    sum_silhouette = 0
    sum_calinski_harabasz = 0
    sum_davies_bouldin = 0
    sum_v_measure = 0
    for j in range(50):
      aggl = AgglomerativeClustering(n_clusters=3, linkage=link).fit(X)
      sum_silhouette += metrics.silhouette_score(X, aggl.labels_, metric='euclidean')
      sum_calinski_harabasz += metrics.calinski_harabasz_score(X, aggl.labels_)
      sum_davies_bouldin += metrics.davies_bouldin_score(X, aggl.labels_)
      sum_v_measure += metrics.v_measure_score(weak_separated_target[i], aggl.labels_)
    aggl_euc_metrics[i].append(sum_v_measure / 50)
    aggl_euc_metrics[i].append(sum_silhouette / 50)
    aggl_euc_metrics[i].append(sum_calinski_harabasz / 50)
    aggl_euc_metrics[i].append(sum_davies_bouldin / 50)

aggl_euc_metrics_table = pd.DataFrame(aggl_euc_metrics,
                                index=pd.Index(["Full", "Nouns", "Nouns and adjs"]),
                                columns=pd.MultiIndex.from_product([["ward", "complete", "average", "single"],['V', 'Sil', 'Cal-Har', 'Dav-Boul']], names=['Metric', 'Linkage']))

aggl_euc_metrics_table.style.set_table_styles([{'selector': 'td', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'th', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'caption', 'props': [('font-size', '24pt')]}]).set_caption("Agglomerative clustering, Euclidean, weak separated")

In [ ]:
aggl_cos_metrics = [[], [], []]

for i in range(3):
  X = datasets[i]
  for link in ["complete", "average", "single"]:
    sum_silhouette = 0
    sum_calinski_harabasz = 0
    sum_davies_bouldin = 0
    sum_v_measure = 0
    for j in range(50):
      aggl = AgglomerativeClustering(n_clusters=3, linkage=link, metric='cosine').fit(X)
      sum_silhouette += metrics.silhouette_score(X, aggl.labels_, metric='euclidean')
      sum_calinski_harabasz += metrics.calinski_harabasz_score(X, aggl.labels_)
      sum_davies_bouldin += metrics.davies_bouldin_score(X, aggl.labels_)
      sum_v_measure += metrics.v_measure_score(weak_separated_target[i], aggl.labels_)
    aggl_cos_metrics[i].append(sum_v_measure / 50)
    aggl_cos_metrics[i].append(sum_silhouette / 50)
    aggl_cos_metrics[i].append(sum_calinski_harabasz / 50)
    aggl_cos_metrics[i].append(sum_davies_bouldin / 50)

aggl_cos_metrics_table = pd.DataFrame(aggl_cos_metrics,
                                index=pd.Index(["Full", "Nouns", "Nouns and adjs"]),
                                columns=pd.MultiIndex.from_product([["complete", "average", "single"],['V', 'Sil', 'Cal-Har', 'Dav-Boul']], names=['Metric', 'Linkage']))

aggl_cos_metrics_table.style.set_table_styles([{'selector': 'td', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'th', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'caption', 'props': [('font-size', '24pt')]}]).set_caption("Agglomerative clustering, cosine, weak separated")

##DistilBERT

###Хорошо разделённые тематики

####Делаем векторизацию

In [ ]:
model_name = 'distilbert-base-uncased'
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertModel.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
well_separated_full_tokens = [tokenizer(text, padding=True, truncation=True, return_tensors="pt") for text in well_separated_full]
well_separated_nouns_tokens = [tokenizer(text, padding=True, truncation=True, return_tensors="pt") for text in well_separated_nouns]
well_separated_nouns_adjs_tokens = [tokenizer(text, padding=True, truncation=True, return_tensors="pt") for text in well_separated_nouns_and_adjs]

X_full_well = []
with torch.no_grad():
  for phrase_tokens in well_separated_full_tokens:
    outputs = model(**phrase_tokens)
    X_full_well.append(np.array(outputs.last_hidden_state.mean(dim=1).flatten()))

X_nouns_well = []
with torch.no_grad():
  for phrase_tokens in well_separated_nouns_tokens:
    outputs = model(**phrase_tokens)
    X_nouns_well.append(np.array(outputs.last_hidden_state.mean(dim=1).flatten()))

X_nouns_and_adjs_well = []
with torch.no_grad():
  for phrase_tokens in well_separated_nouns_adjs_tokens:
    outputs = model(**phrase_tokens)
    X_nouns_and_adjs_well.append(np.array(outputs.last_hidden_state.mean(dim=1).flatten()))

####Выводим метрики

In [ ]:
silhouette = []
calinski_harabasz = []
davies_bouldin = []
v_measure = []
datasets = [X_full_well, X_nouns_well, X_nouns_and_adjs_well]

for j in range(3):
  X = datasets[j]
  sum_silhouette = 0
  sum_calinski_harabasz = 0
  sum_davies_bouldin = 0
  sum_v_measure = 0
  for i in range(50):
    kmeans = KMeans(n_clusters=3, n_init=1).fit(X)
    sum_silhouette += metrics.silhouette_score(X, kmeans.labels_, metric='euclidean')
    sum_calinski_harabasz += metrics.calinski_harabasz_score(X, kmeans.labels_)
    sum_davies_bouldin += metrics.davies_bouldin_score(X, kmeans.labels_)
    sum_v_measure += metrics.v_measure_score(well_separated_target[j], kmeans.labels_)
  v_measure.append(sum_v_measure / 50)
  silhouette.append(sum_silhouette / 50)
  calinski_harabasz.append(sum_calinski_harabasz / 50)
  davies_bouldin.append(sum_davies_bouldin / 50)

kmeans_metrics = pd.DataFrame({
    "V_measure": v_measure,
    "Silhouette": silhouette,
    "Calinski-Harabasz": calinski_harabasz,
    "Davies-Bouldin": davies_bouldin
}, index=pd.Index(["Full", "Nouns", "Nouns and adjs"]))

kmeans_metrics.style.set_table_styles([{'selector': 'td', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'th', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'caption', 'props': [('font-size', '24pt')]}]).set_caption("KMeans, well separated")

,V_measure,Silhouette,Calinski-Harabasz,Davies-Bouldin
Full,0.134095,0.102595,33.360894,2.660068
Nouns,0.122737,0.083129,26.866516,2.732706
Nouns and adjs,0.221745,0.086243,28.525019,2.846022


In [ ]:
aggl_euc_metrics = [[], [], []]

for i in range(3):
  X = datasets[i]
  for link in ["ward", "complete", "average", "single"]:
    sum_silhouette = 0
    sum_calinski_harabasz = 0
    sum_davies_bouldin = 0
    sum_v_measure = 0
    for j in range(50):
      aggl = AgglomerativeClustering(n_clusters=3, linkage=link).fit(X)
      sum_silhouette += metrics.silhouette_score(X, aggl.labels_, metric='euclidean')
      sum_calinski_harabasz += metrics.calinski_harabasz_score(X, aggl.labels_)
      sum_davies_bouldin += metrics.davies_bouldin_score(X, aggl.labels_)
      sum_v_measure += metrics.v_measure_score(well_separated_target[i], aggl.labels_)
    aggl_euc_metrics[i].append(sum_v_measure / 50)
    aggl_euc_metrics[i].append(sum_silhouette / 50)
    aggl_euc_metrics[i].append(sum_calinski_harabasz / 50)
    aggl_euc_metrics[i].append(sum_davies_bouldin / 50)

aggl_euc_metrics_table = pd.DataFrame(aggl_euc_metrics,
                                index=pd.Index(["Full", "Nouns", "Nouns and adjs"]),
                                columns=pd.MultiIndex.from_product([["ward", "complete", "average", "single"],['V', 'Sil', 'Cal-Har', 'Dav-Boul']], names=['Metric', 'Linkage']))

aggl_euc_metrics_table.style.set_table_styles([{'selector': 'td', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'th', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'caption', 'props': [('font-size', '24pt')]}]).set_caption("Agglomerative clustering, Euclidean, well separated")

In [ ]:
aggl_cos_metrics = [[], [], []]

for i in range(3):
  X = datasets[i]
  for link in ["complete", "average", "single"]:
    sum_silhouette = 0
    sum_calinski_harabasz = 0
    sum_davies_bouldin = 0
    sum_v_measure = 0
    for j in range(50):
      aggl = AgglomerativeClustering(n_clusters=3, linkage=link, metric='cosine').fit(X)
      sum_silhouette += metrics.silhouette_score(X, aggl.labels_, metric='euclidean')
      sum_calinski_harabasz += metrics.calinski_harabasz_score(X, aggl.labels_)
      sum_davies_bouldin += metrics.davies_bouldin_score(X, aggl.labels_)
      sum_v_measure += metrics.v_measure_score(well_separated_target[i], aggl.labels_)
    aggl_cos_metrics[i].append(sum_v_measure / 50)
    aggl_cos_metrics[i].append(sum_silhouette / 50)
    aggl_cos_metrics[i].append(sum_calinski_harabasz / 50)
    aggl_cos_metrics[i].append(sum_davies_bouldin / 50)

aggl_cos_metrics_table = pd.DataFrame(aggl_cos_metrics,
                                index=pd.Index(["Full", "Nouns", "Nouns and adjs"]),
                                columns=pd.MultiIndex.from_product([["complete", "average", "single"],['V', 'Sil', 'Cal-Har', 'Dav-Boul']], names=['Metric', 'Linkage']))

aggl_cos_metrics_table.style.set_table_styles([{'selector': 'td', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'th', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'caption', 'props': [('font-size', '24pt')]}]).set_caption("Agglomerative clustering, cosine, well separated")

###Плохо разделённые тематики

####Делаем векторизацию

In [ ]:
weak_separated_full_tokens = [tokenizer(text, padding=True, truncation=True, return_tensors="pt") for text in weak_separated_full]
weak_separated_nouns_tokens = [tokenizer(text, padding=True, truncation=True, return_tensors="pt") for text in weak_separated_nouns]
weak_separated_nouns_adjs_tokens = [tokenizer(text, padding=True, truncation=True, return_tensors="pt") for text in weak_separated_nouns_and_adjs]

X_full_weak = []
with torch.no_grad():
  for phrase_tokens in weak_separated_full_tokens:
    outputs = model(**phrase_tokens)
    X_full_weak.append(np.array(outputs.last_hidden_state.mean(dim=1).flatten()))

X_nouns_weak = []
with torch.no_grad():
  for phrase_tokens in weak_separated_nouns_tokens:
    outputs = model(**phrase_tokens)
    X_nouns_weak.append(np.array(outputs.last_hidden_state.mean(dim=1).flatten()))

X_nouns_and_adjs_weak = []
with torch.no_grad():
  for phrase_tokens in weak_separated_nouns_adjs_tokens:
    outputs = model(**phrase_tokens)
    X_nouns_and_adjs_weak.append(np.array(outputs.last_hidden_state.mean(dim=1).flatten()))

####Выводим метрики

In [ ]:
silhouette = []
calinski_harabasz = []
davies_bouldin = []
v_measure = []
datasets = [X_full_weak, X_nouns_weak, X_nouns_and_adjs_weak]

for j in range(3):
  X = datasets[j]
  sum_silhouette = 0
  sum_calinski_harabasz = 0
  sum_davies_bouldin = 0
  sum_v_measure = 0
  for i in range(50):
    kmeans = KMeans(n_clusters=3, n_init=1).fit(X)
    sum_silhouette += metrics.silhouette_score(X, kmeans.labels_, metric='euclidean')
    sum_calinski_harabasz += metrics.calinski_harabasz_score(X, kmeans.labels_)
    sum_davies_bouldin += metrics.davies_bouldin_score(X, kmeans.labels_)
    sum_v_measure += metrics.v_measure_score(weak_separated_target[j], kmeans.labels_)
  v_measure.append(sum_v_measure / 50)
  silhouette.append(sum_silhouette / 50)
  calinski_harabasz.append(sum_calinski_harabasz / 50)
  davies_bouldin.append(sum_davies_bouldin / 50)

kmeans_metrics = pd.DataFrame({
    "V_measure": v_measure,
    "Silhouette": silhouette,
    "Calinski-Harabasz": calinski_harabasz,
    "Davies-Bouldin": davies_bouldin
}, index=pd.Index(["Full", "Nouns", "Nouns and adjs"]))

kmeans_metrics.style.set_table_styles([{'selector': 'td', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'th', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'caption', 'props': [('font-size', '24pt')]}]).set_caption("KMeans, weak separated")

,V_measure,Silhouette,Calinski-Harabasz,Davies-Bouldin
Full,0.041329,0.127267,40.584710,2.446486
Nouns,0.053887,0.121227,40.811004,1.973898
Nouns and adjs,0.042323,0.138006,43.661180,1.844568


In [ ]:
aggl_euc_metrics = [[], [], []]

for i in range(3):
  X = datasets[i]
  for link in ["ward", "complete", "average", "single"]:
    sum_silhouette = 0
    sum_calinski_harabasz = 0
    sum_davies_bouldin = 0
    sum_v_measure = 0
    for j in range(50):
      aggl = AgglomerativeClustering(n_clusters=3, linkage=link).fit(X)
      sum_silhouette += metrics.silhouette_score(X, aggl.labels_, metric='euclidean')
      sum_calinski_harabasz += metrics.calinski_harabasz_score(X, aggl.labels_)
      sum_davies_bouldin += metrics.davies_bouldin_score(X, aggl.labels_)
      sum_v_measure += metrics.v_measure_score(weak_separated_target[i], aggl.labels_)
    aggl_euc_metrics[i].append(sum_v_measure / 50)
    aggl_euc_metrics[i].append(sum_silhouette / 50)
    aggl_euc_metrics[i].append(sum_calinski_harabasz / 50)
    aggl_euc_metrics[i].append(sum_davies_bouldin / 50)

aggl_euc_metrics_table = pd.DataFrame(aggl_euc_metrics,
                                index=pd.Index(["Full", "Nouns", "Nouns and adjs"]),
                                columns=pd.MultiIndex.from_product([["ward", "complete", "average", "single"],['V', 'Sil', 'Cal-Har', 'Dav-Boul']], names=['Metric', 'Linkage']))

aggl_euc_metrics_table.style.set_table_styles([{'selector': 'td', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'th', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'caption', 'props': [('font-size', '24pt')]}]).set_caption("Agglomerative clustering, Euclidean, weak separated")

In [ ]:
aggl_cos_metrics = [[], [], []]

for i in range(3):
  X = datasets[i]
  for link in ["complete", "average", "single"]:
    sum_silhouette = 0
    sum_calinski_harabasz = 0
    sum_davies_bouldin = 0
    sum_v_measure = 0
    for j in range(50):
      aggl = AgglomerativeClustering(n_clusters=3, linkage=link, metric='cosine').fit(X)
      sum_silhouette += metrics.silhouette_score(X, aggl.labels_, metric='euclidean')
      sum_calinski_harabasz += metrics.calinski_harabasz_score(X, aggl.labels_)
      sum_davies_bouldin += metrics.davies_bouldin_score(X, aggl.labels_)
      sum_v_measure += metrics.v_measure_score(weak_separated_target[i], aggl.labels_)
    aggl_cos_metrics[i].append(sum_v_measure / 50)
    aggl_cos_metrics[i].append(sum_silhouette / 50)
    aggl_cos_metrics[i].append(sum_calinski_harabasz / 50)
    aggl_cos_metrics[i].append(sum_davies_bouldin / 50)

aggl_cos_metrics_table = pd.DataFrame(aggl_cos_metrics,
                                index=pd.Index(["Full", "Nouns", "Nouns and adjs"]),
                                columns=pd.MultiIndex.from_product([["complete", "average", "single"],['V', 'Sil', 'Cal-Har', 'Dav-Boul']], names=['Metric', 'Linkage']))

aggl_cos_metrics_table.style.set_table_styles([{'selector': 'td', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'th', 'props': [('font-size', '12pt'),('border-style','solid'),('border-width','1px')]},
                                          {'selector': 'caption', 'props': [('font-size', '24pt')]}]).set_caption("Agglomerative clustering, cosine, weak separated")